# Sistemas de Recomendação - Método 02

Neste notebook vamos desenvolver outro simples sistema de recomendação usando Python, Pandas e Scikit Learn. Nosso objetivo é desenvolver um sistema um sistema basico capaz de realizar sugestões de filmes baseados nas avaliações dos usuários.

Para esse exemplo usaremos o arquivo `ml-latest-small.zip`, o qual contêm um conjunto de dados reduzido.

------
## Importa Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Os dados utilizados são provenientes da plataforma [MovieLens](https://grouplens.org/datasets/movielens/). Primeiramente vamos entender o conteúdo dos dados no arquivo `ratings.csv` responsávle por armazenar as notas dos usuários.

In [2]:
df_notas = pd.read_csv('../datasets/ml-latest-small/ratings.csv')

In [3]:
df_notas.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Agora vamos explorar o arquivo `movies.csv` o qual contem o nome e outras informações importantes dos filmes contidos na base de dados `ratings.csv`

In [4]:
df_filmes = pd.read_csv('../datasets/ml-latest-small/movies.csv')

In [5]:
df_filmes.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


------
## Sistemas de Recomendação

Vamos explorar a utilização de um sistema de recomendação para filmes baseada na classificação dada por outros usuários. Para uma comprenssão mais detalhada do seu funcioamento leia o artigo presente [neste link](https://www.ibm.com/developerworks/br/local/data/sistemas_recomendacao/index.html).

<img width='500px' src='https://www.ibm.com/developerworks/br/local/data/sistemas_recomendacao/image001_s.jpg' />

Quando trata-se de sistemas de recomendação podemos citar três tipos:
* **Baseado em Conteúdo:** O sistema recomenda ao usuário produtos que sejam semelhantes ao que ele preferiu no passado. Em um cenário de recomendação de filmes, por exemplo, um usuário que, assiste e gosta do filme "Matrix" teria recomendações do gênero ação e ficção científica.
* **Filtragem Colaborativa:** O sistema recomenda itens baseado em em gosto de outros usuários. A regra baseia-se em: "Se um usuário gostou de A e de B, um outro usuário que gostou de A também pode gostar de B"
* **Sistemas Hibrídos:** Consiste em um sistema que combina as duas abordagens mencionadas, tentando fortificá-las e superar suas desvantagens.

-----
### Método 2

Agora testaremos outro método. Para isso usaremos a librarie [Surprise](http://surpriselib.com/) a qual detêm inumeros algoritmos de previsão para sistemas de recomendação, como: SVD, NMF, SlopeOne, KNN, dentre outros. Para esse exemplo usaremos o algoritmo o famoso SVD, para compreender melhor o seu funcionamento acesse [este link](https://medium.com/@m_n_malaeb/singular-value-decomposition-svd-in-recommender-systems-for-non-math-statistics-programming-4a622de653e9).

In [6]:
from collections import defaultdict
from surprise import Reader, Dataset
from surprise import SVD
from surprise.accuracy import rmse
from surprise.model_selection import train_test_split

Vamos criar a estrutura de dados légivel do surprise.

In [7]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df_notas[['userId', 'movieId', 'rating']], reader)

data

Feito isso, vamos atribir o objeto SVD a uma variável algo. Além disso, definiremos (empiricamente) alguns parâmetros para melhorar a performace do algorimo. Sinto a vontade para alterar esses parâmetros.

In [8]:
# Parâmetros para o objeto SVD
factors = 35
epochs = 25
lr_value = 0.008
reg_value = 0.08

In [9]:
algo = SVD(n_factors=factors, n_epochs=epochs, lr_all=lr_value, reg_all=reg_value)

Vamos dividir nosso dados em treino e teste e estimar o RMSE do nosso modelo.

In [10]:
trainset, testset = train_test_split(data, test_size=0.25) 

In [11]:
predictions = algo.fit(trainset).test(testset)

In [12]:
rmse(predictions)

RMSE: 0.8700


0.86996462335584

Vamos checar o conteúdo das nossas predições.

In [13]:
predictions[0:5]

[Prediction(uid=357, iid=33794, r_ui=4.0, est=4.0892002002777215, details={'was_impossible': False}),
 Prediction(uid=603, iid=2575, r_ui=4.0, est=3.271860646915173, details={'was_impossible': False}),
 Prediction(uid=357, iid=1080, r_ui=5.0, est=4.165774334877863, details={'was_impossible': False}),
 Prediction(uid=508, iid=3317, r_ui=2.0, est=2.543230078753657, details={'was_impossible': False}),
 Prediction(uid=180, iid=2723, r_ui=3.0, est=3.2343141254722223, details={'was_impossible': False})]

Notamos que foram geradas diversas variáveis que possuem sua documentação no [site do surprise](https://surprise.readthedocs.io/en/stable/predictions_module.html). Dessa forma, vamos verificar quão boas ou ruins são nossas previsões. A função a seguir criará uma DataFrame que terá as seguintes colunas.

* `UID`: id do usuário
* `iid`: id do filme
* `Rui`: a classificação dada pelo usuário
* `est`: classificação estimada pelo modelo
* `Iu`: nenhum dos itens classificados pelo usuário
* `UI`: número de usuários que classificaram este filme
* `err`: diferença abs entre a classificação prevista e a classificação real.

In [14]:
def get_Iu(uid):
    """ 
    args: 
      uid: o id do usuárop
    returns: 
      o número de filmes classificados pelo usuário
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # usuário não está contido no dataset de treino
        return 0
    
def get_Ui(iid):
    """ 
    args:
      iid: o id do filme
    returns:
      o número de usuários que avaliaram esse filme
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0

In [15]:
# Cria um dataframe com o resultados das predições
df_predictions = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])

In [16]:
# Calcula os demais parâmetros
df_predictions['Iu'] = df_predictions['uid'].apply(get_Iu)
df_predictions['Ui'] = df_predictions['iid'].apply(get_Ui)
df_predictions['err'] = abs(df_predictions['est'] - df_predictions['rui'])

In [17]:
df_predictions.head()

,uid,iid,rui,est,details,Iu,Ui,err
0,357,33794,4.0,4.089200,{'was_impossible': False},289,90,0.089200
1,603,2575,4.0,3.271861,{'was_impossible': False},706,3,0.728139
2,357,1080,5.0,4.165774,{'was_impossible': False},289,62,0.834226
3,508,3317,2.0,2.543230,{'was_impossible': False},19,12,0.543230
4,180,2723,3.0,3.234314,{'was_impossible': False},19,29,0.234314


Agora vamos checar algumas recomendações baseadas nas avaliações realizadas pelos usuários.

In [18]:
# Não divide o conjunto de dados em teste e treino
trainset = data.build_full_trainset()

In [19]:
algo = SVD(n_factors=factors, n_epochs=epochs, lr_all=lr_value, reg_all=reg_value)
algo.fit(trainset)

In [20]:
# Prevê as classificações para todos os pares (u, i) que NÃO estão no conjunto de treinamento.
testset = trainset.build_anti_testset()

# Prevê as notas para o conjunto de teste
predictions = algo.test(testset)

In [21]:
def get_all_predictions(predictions):
    """ 
    args:
      predictions: as predições de cada usuário
    returns:
      top_n: dict com as predições ordenadas
    """
    # Inicialmente vamos mapear as previsões de cada usuário
    top_n = defaultdict(list)
    
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Então vamos ordenar os usuários
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)

    return top_n

In [22]:
all_pred = get_all_predictions(predictions)

Agora vamos identificar as 5 principais recomendações baseado no histórico do usuário

In [23]:
#To get top 4 reommendation
n = 5

for uid, user_ratings in all_pred.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    all_pred[uid] = user_ratings[:n]

In [24]:
tmp = pd.DataFrame.from_dict(all_pred)
tmp_transpose = tmp.transpose()

In [25]:
tmp_transpose.head()

,0,1,2,3,4
1,"(318, 5.0)","(720, 5.0)","(912, 5.0)","(930, 5.0)","(2324, 5.0)"
2,"(951, 4.457544638028725)","(177593, 4.450745916955722)","(3451, 4.442460454899721)","(1041, 4.423438864916136)","(1104, 4.410398872572914)"
3,"(899, 3.7388597499270793)","(2360, 3.5786611575659344)","(122926, 3.5136999199969217)","(58559, 3.511603525356694)","(56782, 3.5100842254704685)"
4,"(47629, 4.38161812265284)","(1041, 4.3517921104075725)","(5747, 4.346552826348474)","(8132, 4.31132702609768)","(1178, 4.305531741501966)"
5,"(177593, 4.454752474654791)","(750, 4.390112701610291)","(3266, 4.321130948950757)","(27156, 4.299915506938349)","(1041, 4.288654321694549)"


Agora, temos um DataFrame que consiste nos 5 principais filmes recomendados para todos os usuários.
Vamos tentar um exemplo e encontrar recomendações para o usuário 67.

In [26]:
def get_predictions(user_id):
    results = tmp_transpose.loc[user_id]
    return results

In [27]:
user_id = 67
results = get_predictions(user_id)
results

0    (177593, 4.491415882796029)
1      (3451, 4.482683390138332)
2      (1104, 4.426604501871211)
3      (1223, 4.412225473933734)
4      (1204, 4.389667369302512)
Name: 67, dtype: object

Vamos extrair o índice dos filmes recomendados.

In [28]:
recommended_movie_ids = []

for i in range(0, n):
    recommended_movie_ids.append(results[i][0])

recommended_movie_ids

[177593, 3451, 1104, 1223, 1204]

Vamos encontrar outros detalhes sobre os filmes recomendados.

In [29]:
recommended_movie = df_filmes[df_filmes['movieId'].isin(recommended_movie_ids)]
recommended_movie

,movieId,title,genres
841,1104,"Streetcar Named Desire, A (1951)",Drama
906,1204,Lawrence of Arabia (1962),Adventure|Drama|War
924,1223,"Grand Day Out with Wallace and Gromit, A (1989)",Adventure|Animation|Children|Comedy|Sci-Fi
2582,3451,Guess Who's Coming to Dinner (1967),Drama
9618,177593,"Three Billboards Outside Ebbing, Missouri (2017)",Crime|Drama


Dessa forma temos as principais indicações para qualquer usuário específico